# ML Pipeline Preparation

Follow the instructions below to help you create your ML pipeline.

### 1. Import libraries and load data from database.

- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y


In [1]:
# import libraries
from sqlalchemy import create_engine
import re
from typing import Iterable

import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [2]:
# load data from database
engine = create_engine("sqlite:///../data/disaster/disaster_response.db")
df = pd.read_sql("disaster_messages", engine)
X = df["message"]
category_columns = [
    col for col in df.columns if sorted(df[col].dropna().unique()) == [0, 1]
]
y = df[category_columns]

In [5]:
df = pd.read_csv("../data/models/train_test_metrics.csv", index_col=0, header=[0, 1])

In [12]:
df.transpose().unstack(level=1)

balanced_accuracy            f1_score            \
                                    test     train      test     train   
aid_centers                     0.500000  0.946429  0.000000  0.943396   
aid_related                     0.722869  0.997441  0.648678  0.997240   
buildings                       0.521966  0.984234  0.084291  0.983545   
clothing                        0.539280  0.983180  0.142857  0.982893   
cold                            0.504407  0.975962  0.017699  0.975369   
death                           0.558652  0.988033  0.208494  0.987888   
direct_report                   0.675861  0.992246  0.507757  0.991634   
earthquake                      0.771211  0.996465  0.676093  0.994413   
electricity                     0.499903  0.977011  0.000000  0.976471   
fire                            0.500000  0.971429  0.000000  0.970588   
floods                          0.704380  0.989764  0.563694  0.989393   
food                            0.713801  0.995924  0.579611  0.995908   
hospitals                       0.500000  0.971491  0.000000  0.970655   
infrastructure_related          0.499693  0.975055  0.000000  0.974417   
medical_help                    0.514530  0.985463  0.058296  0.985249   
medical_products                0.525189  0.979621  0.095941  0.979197   
military                        0.522860  0.983942  0.086957  0.983680   
missing_people                  0.500000  0.963983  0.000000  0.962637   
money                           0.517596  0.978659  0.066667  0.978193   
offer                           0.500000  0.978022  0.000000  0.977528   
other_aid                       0.528032  0.985335  0.114667  0.984051   
other_infrastructure            0.499700  0.970362  0.000000  0.969457   
other_weather                   0.510272  0.977997  0.041237  0.977071   
refugees                        0.517455  0.980114  0.067416  0.979710   
related                         0.703921  0.994548  0.890588  0.997756   
request                         0.729646  0.992597  0.602410  0.992216   
search_and_rescue               0.509705  0.972052  0.038462  0.970404   
security                        0.499903  0.960398  0.000000  0.957476   
shelter                         0.634284  0.995701  0.416667  0.995683   
shops                           0.500000  0.961905  0.000000  0.960396   
storm                           0.663587  0.993491  0.470752  0.992985   
tools                           0.500000  0.951493  0.000000  0.949020   
transport                       0.508203  0.970225  0.032787  0.968817   
water                           0.629987  0.992984  0.401961  0.992934   
weather_related                 0.763076  0.996047  0.678571  0.995503   

                       precision              recall            
                            test     train      test     train  
aid_centers             0.000000  1.000000  0.000000  0.892857  
aid_related             0.765273  0.998618  0.562914  0.995865  
buildings               0.785714  0.999045  0.044534  0.968519  
clothing                0.750000  1.000000  0.078947  0.966361  
cold                    0.500000  1.000000  0.009009  0.951923  
death                   0.900000  1.000000  0.117904  0.976067  
direct_report           0.789474  0.998513  0.374220  0.984848  
earthquake              0.876667  0.995425  0.550209  0.993404  
electricity             0.000000  1.000000  0.000000  0.954023  
fire                    0.000000  1.000000  0.000000  0.942857  
floods                  0.885000  0.999405  0.413551  0.979580  
food                    0.863481  1.000000  0.436207  0.991849  
hospitals               0.000000  1.000000  0.000000  0.942982  
infrastructure_related  0.000000  1.000000  0.000000  0.950110  
medical_help            0.650000  1.000000  0.030516  0.970927  
medical_products        0.764706  1.000000  0.051181  0.959242  
military                0.666667  1.000000  0.046512  0.967883  
missing_people          0.000000  1.000000  0.000

In [16]:
df.xs("test", axis=1, level=1, drop_level=False).mean(axis=1)

balanced_accuracy    0.571142
f1_score             0.214073
precision            0.502802
recall               0.164637
dtype: float64

### 2. Write a tokenization function to process your text data


In [ ]:
def tokenize(text: str) -> Iterable[str]:
    """Pre-process text into lemmatized tokens.

    Args:
        text: Text to process.

    Returns:
        Iterable of lemmatized tokens.
    """
    # 0. Setup
    url_regex = (
        "http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
    )
    non_alphanum_regex = "^[a-z0-9]"

    # 1. Replace URLs
    text = re.sub(url_regex, " ", text)

    # 2. Replace non-alphanumeric characters
    text = re.sub(non_alphanum_regex, " ", text.lower())

    # 3. Tokenize and lemmatize text
    lemmatizer = WordNetLemmatizer()
    tokens = [
        lemmatizer.lemmatize(lemmatizer.lemmatize(token), pos="v")
        for token in word_tokenize(text)
        if token not in stopwords.words("english")
    ]

    return tokens

### 3. Build a machine learning pipeline

This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

pipeline = Pipeline(
    [("vect", TfidfVectorizer(tokeni)), ("clf", RandomForestClassifier())]
)

# https://radimrehurek.com/gensim_3.8.3/sklearn_api/d2vmodel.html

### 4. Train pipeline

- Split data into train and test sets
- Train pipeline


In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y)

# train classifier
pipeline.fit(X_train, y_train)

### 5. Test your model

Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.


In [ ]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report


print(classification_report(y_test, y_pred))

In [ ]:
y_test.iloc[:, 0]

### 6. Improve your model

Use grid search to find better parameters.


In [ ]:
parameters = 

cv = 

### 7. Test your model

Show the accuracy, precision, and recall of the tuned model.

Since this project focuses on code quality, process, and pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!


### 8. Try improving your model further. Here are a few ideas:

- try other machine learning algorithms
- add other features besides the TF-IDF


### 9. Export your model as a pickle file


### 10. Use this notebook to complete `train.py`

Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.
